In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
import tqdm

# Paper recap

x1 and x2 are sequences generated from constrained random walks.  x1 (x2) changes with a prob per unit time of 
0.65 (0.99) by a random amount uniformly distributed between -0.5 and 0.5 (-1 and 1). The modulus of x is then 
taken to ensure a positive sequence. Initial v are randomly selected from a unif fistrbution between -1 and 1. 

v1 ans v2 are unif randomly selected between -1 and +1.

100 temporal patterns are simulated numerically with different sequences for x1 and x2, and different inital v1 and v2. Tau goes from 0.5 to 0.8 with dt=0.1

Network: 8 hidden nodes and trained on 50 of the processes. Results shown on 50 of the processes not seen during the training. Set alpha=0.01 and beta=40 000. 

Temporak pattern: network produces sequence v(1) ... v(T) given initial conditions v(0) and ext input x(0), ..., x(T-1) and time steps. 

The neural network takes as input v(i) and x(i), and computes the function dv/dt=F(v,x).
First layer: inputs v(i) and x(i)
Second layer: 8 hidden nodes with tanh function 
Output layr: dv/dt no activation function 

# Generate the data 

### The diff equation

In [2]:
def diff_equ(t, V, x1, x2):
    #x1, x2 =tf.split(X[tf.cast(t, tf.int32)],1)
    v1, v2 = V 
    dv1= x1[tf.dtypes.cast(tf.round(t), tf.int32)] -2*v1 + 8*v2 -x1[tf.dtypes.cast(tf.round(t), tf.int32)]*v1
    dv2= x2[tf.dtypes.cast(tf.round(t), tf.int32)] -5*v1 + v2 -x2[tf.dtypes.cast(tf.round(t), tf.int32)]*v2
    dV=np.array([dv1, dv2])
    return dV   


### Generate x value sequences for 0... T-1

In [3]:
def gen_x(T):
    #initial values 
    x1=np.absolute(np.array([np.random.uniform(low=-0.5, high=0.5)]))
    x2=np.absolute(np.array([np.random.uniform(low=-1.0, high=1.0)]))
    #x1=np.absolute(np.array([[np.random.uniform(low=-0.5, high=0.5)]]))
    #x2=np.absolute(np.array([[np.random.uniform(low=-1.0, high=1.0)]]))
    for i in range(T): 
        #generate a bernouilli according to PD
        p1 = np.random.binomial(1,p=0.65)
        p2 = np.random.binomial(1,p=0.99)
        u1=np.absolute(np.array([np.random.uniform(low=-0.5, high=0.5)]))
        u2=np.absolute(np.array([np.random.uniform(low=-1.0, high=1.0)]))
        x1=np.append(x1, x1[-1]+p1*u1, axis=0)
        x2=np.append(x2, x2[-1]+p2*u2, axis=0)
    #X=np.hstack((x1,x2))
    return x1, x2

In [4]:
#Regularisation para see equ.(24)
alpha=0.01
#Loss function para´see equ.(10)
beta=40000

ODESolver = tfp.math.ode.DormandPrince()

# initial conditions and time grid
t_initial = 0 # initial time
t_final = 8 # final time
n_times = 80 # number of time steps to churn out solution for
times = np.linspace(t_initial, t_final, n_times).astype(np.float32) 

#v1=np.array([[np.random.uniform(low=-1.0, high=1.0)]])
#v2=np.array([[np.random.uniform(low=-1.0, high=1.0)]])
#v_initial = np.array([v1, v2]).astype(np.float32) # initial state vector

v1=np.random.uniform(low=-1.0, high=1.0)
v2=np.random.uniform(low=-1.0, high=1.0)

v_initial = np.array([v1, v2]).astype(np.float32)

x1, x2=gen_x(n_times)
# integrate the ODE
results = ODESolver.solve(diff_equ, # system of ODEs (gradient function)
                                   t_initial, # initial time
                                   v_initial, # initial state
                                   solution_times=times,
                                   constants={'x1': x1.astype(np.float32),'x2': x2.astype(np.float32)}) # time grid to spit out solutions for

# extract results for the state solutions v(t)
data = tf.cast(tf.stack(results.states, axis=-1), tf.float32)

In [5]:
v_initial

array([-0.12609522, -0.3306193 ], dtype=float32)

In [27]:
dt=0.1
def euler(v, dv):
    dv1, dv2=dv
    new_v1=v1+dv1*dt
    new_v2=v2+dv2*dt
    V=tf.stack([new_v1, new_v2])
    return V

In [28]:
# weights and biases:
n_state = 2
n_ext=2
n_hidden = 8
W1_v = tf.Variable(tf.random.normal([(n_state), n_hidden], 0, 0.1),  trainable=True)
W1_x=  tf.Variable(tf.random.normal([(n_ext), n_hidden], 0, 0.1),  trainable=True)
b1 = tf.Variable(tf.random.normal([n_hidden], 0, 0.1),  trainable=True)
W2 = tf.Variable(tf.random.normal([n_hidden, n_state], 0, 0.1),  trainable=True)
b2 = tf.Variable(tf.random.normal([n_state], 0, 0.1),  trainable=True)

def dvdt_nn(t,v,x):
    output=tf.matmul(tf.expand_dims(v, axis=0), W1_v)+tf.matmul(tf.expand_dims(x, axis=0), W1_x)+ b1
    dvdt = tf.matmul(tf.math.tanh(output), W2) + b2   
    return tf.squeeze(dvdt, axis=0)

For a given epoch: 
    Error: e=v-T where T is the target 
    Total error: 1/2 \sum \beta(e)^2

### BJ step 

In [29]:
x=np.array([x1,x2]).astype(np.float32)
x=np.transpose(x)

dvdt_nn(0, v_initial, x[0])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.03633432, -0.14558354], dtype=float32)>

In [30]:

predictions

NameError: name 'predictions' is not defined

In [31]:
# choose optimizer
optimizer = tf.keras.optimizers.Adam(lr = 1e-3)

# training step: computes predictions, compute MSE between predictions and data, computes gradients and applies them to the parameters
def training_step():
    
    # start the gradient tape: this records all operations in a way that gradients can be taken
    with tf.GradientTape() as tape:
        tape.watch([W1_v, W1_x, W2, b1, b2])
        predictions=[]
        pred_v = v_initial
        for i in range(len(times)):
            net=dvdt_nn(times[i], pred_v, x[i])
            pred_v = euler(pred_v, net)
            predictions.append(pred_v)
        predictions=tf.stack(predictions)
        
        # calculate loss
        error=data-tf.transpose(predictions) 
        loss=tf.reduce_mean(tf.square(error))      
        
    # calculate gradients
    gradients = tape.gradient(loss, tape.watched_variables())
    
    # make gradient step
    optimizer.apply_gradients(zip(gradients, tape.watched_variables()))   
    
    return loss

In [32]:
# list to keep the loss values for plotting later (to see if it's converged)
losses = []

# number of trainine epochs
n_epochs = 200

# progressbar
pbar = tqdm.tqdm_notebook(total = n_epochs, desc = "Optimizing")
pbar.set_postfix(ordered_dict={"loss":None}, refresh=True)

# train it! loop over epochs, doing a gradient update each step
for i in range(n_epochs):
    
    loss = training_step()
    losses.append(loss.numpy())
    
    # update progressbar
    pbar.update()
    pbar.set_postfix(ordered_dict={"loss":loss.numpy()}, refresh=True)

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


KeyboardInterrupt: 

In [17]:
data

<tf.Tensor: shape=(2, 80), dtype=float32, numpy=
array([[-0.12609522, -0.27173555, -0.25115174, -0.09910691,  0.10850728,
         0.29274505,  0.42005032,  0.45026696,  0.39502618,  0.29339138,
         0.19192702,  0.12630874,  0.11122584,  0.14027728,  0.19305354,
         0.24505073,  0.29613233,  0.33318233,  0.3455403 ,  0.33575672,
         0.31312114,  0.28899068,  0.27228394,  0.26679718,  0.26986635,
         0.3067457 ,  0.35291827,  0.3819416 ,  0.38931745,  0.3794882 ,
         0.36137876,  0.3439673 ,  0.33304894,  0.33029023,  0.33300513,
         0.3460072 ,  0.39590383,  0.45898718,  0.50764894,  0.52997184,
         0.52805024,  0.51150894,  0.49179623,  0.47693747,  0.4700289 ,
         0.47250703,  0.49904707,  0.5379966 ,  0.5690656 ,  0.5852802 ,
         0.5865511 ,  0.57853806,  0.56784916,  0.55922127,  0.5552815 ,
         0.5829935 ,  0.64164513,  0.69462067,  0.725401  ,  0.7332347 ,
         0.7261722 ,  0.7137735 ,  0.7032021 ,  0.6971952 ,  0.6958306 ,
  

In [18]:
predictions

NameError: name 'predictions' is not defined